In [1]:
# Configuration

# days used in calculating ratios
DAYS_RATIO = [3, 5, 10, 15, 30, 60, 120, 250]

# risk free rate
RISK_FREE_RATE = 0.04

In [2]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [3]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
# print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DEST_DIR) / f'{date_str}_df_OHLCV_clean.parquet'
dest_path = Path(DEST_DIR) / f'{date_str}_df_perf_ratios.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-23
DEST_DIR: ..\data

source_path: ..\data\2025-04-23_df_OHLCV_clean.parquet
dest_path: ..\data\2025-04-23_df_perf_ratios.parquet


In [4]:
import pandas as pd

# Get tickers from df_finviz.pkl file
df_finviz = pd.read_parquet(f'..\data\{date_str}_df_finviz.parquet')
tickers = df_finviz.index.to_list()

In [5]:
import pandas as pd

pd.set_option('display.width', 1000) 

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# # Display the first few rows of the DataFrame to verify
# df = df[['Adj Close']].copy()
display(df)

Open    High     Low   Close  Adj Close   Volume    Adj Open    Adj High     Adj Low
Symbol Date                                                                                              
RY     2025-04-23  119.27  119.86  117.42  117.79     117.79   976958  119.270000  119.860000  117.420000
       2025-04-22  115.82  117.97  115.41  117.97     117.97  5486100  115.820000  117.970000  115.410000
       2025-04-21  116.00  116.60  114.63  115.21     115.21   600100  116.000000  116.600000  114.630000
       2025-04-17  115.53  116.71  115.45  116.24     116.24  1090900  115.530000  116.710000  115.450000
       2025-04-16  115.21  116.24  114.25  115.62     115.62   935000  115.210000  116.240000  114.250000
...                   ...     ...     ...     ...        ...      ...         ...         ...         ...
TPR    2024-04-30   40.55   40.74   39.90   39.92      38.88  5029500   39.493587   39.678637   38.860521
       2024-04-29   40.31   41.01   40.18   40.98      39.91  4161300   39.257494   39.939217   39.130888
       2024-04-26   39.54   40.38   39.40   40.05      39.01  3329600   38.513243   39.331431   38.376879
       2024-04-25   39.85   39.91   38.95   39.39      38.36  4311000   38.807972   38.866403   37.931505
       2024-04-24   39.39   40.07   39.03   40.03      38.99  4829300   38.366628   39.028961   38.015981

[384250 rows x 9 columns]

In [6]:
import pandas as pd

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
display(df)

Adj Close
Symbol Date                 
RY     2025-04-23     117.79
       2025-04-22     117.97
       2025-04-21     115.21
       2025-04-17     116.24
       2025-04-16     115.62
...                      ...
TPR    2024-04-30      38.88
       2024-04-29      39.91
       2024-04-26      39.01
       2024-04-25      38.36
       2024-04-24      38.99

[384250 rows x 1 columns]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 384250 entries, ('RY', Timestamp('2025-04-23 00:00:00')) to ('TPR', Timestamp('2024-04-24 00:00:00'))
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Close  384250 non-null  float64
dtypes: float64(1)
memory usage: 4.5+ MB


In [8]:
import pandas as pd

def select_tickers_data(df, tickers):
  """
  Selects data for a list of tickers from a DataFrame with a MultiIndex
  where the first level is the ticker and the second level is the date.

  Args:
    df (pd.DataFrame): The input DataFrame with a MultiIndex.
    tickers (list): A list of ticker symbols to select.

  Returns:
    pd.DataFrame: A DataFrame containing only the data for the specified tickers.
                  Rows for tickers not found will not be included.
  """
  valid_tickers = [t for t in tickers if t in df.index.get_level_values(0)]  # Filter out the tickers that do not exist

  if not valid_tickers:
    print("No valid tickers found in the DataFrame. Returning an empty DataFrame.")
    return pd.DataFrame()

  try:
    ticker_df = df.loc[valid_tickers]
    return ticker_df
  except KeyError as e:
    print(f"KeyError after filtering valid tickers: {e}") #Added to help debugging.
    return pd.DataFrame()


# Example usage:
# ticker_list = ["UBS", "AAPL", "MSFT", "GEV"]  # Include some valid and invalid tickers
selected_data = select_tickers_data(df, tickers)
print(selected_data)

                   Adj Close
Symbol Date                 
AAPL   2025-04-23     204.60
       2025-04-22     199.74
       2025-04-21     193.16
       2025-04-17     196.98
       2025-04-16     194.27
...                      ...
TFI    2024-04-30      44.25
       2024-04-29      44.32
       2024-04-26      44.24
       2024-04-25      44.23
       2024-04-24      44.41

[384250 rows x 1 columns]


In [9]:
# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

import sys

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

import utils

list_dfs = utils.get_latest_dfs(selected_data, DAYS_RATIO)

In [10]:
import numpy as np

all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        # Suppress division warnings just for this calculation
        with np.errstate(divide='ignore', invalid='ignore'):
            result_df = utils.analyze_stock(_df, ticker, risk_free_rate=RISK_FREE_RATE)
        
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")


Combined performance metrics DataFrame:
        Sharpe 3d  Sortino 3d  Omega 3d  Sharpe 5d  Sortino 5d  Omega 5d  Sharpe 10d  Sortino 10d  Omega 10d  Sharpe 15d  Sortino 15d  Omega 15d  Sharpe 30d  Sortino 30d  Omega 30d  Sharpe 60d  Sortino 60d  Omega 60d  Sharpe 120d  Sortino 120d  Omega 120d  Sharpe 250d  Sortino 250d  Omega 250d
A      126.085003         inf       NaN   4.034383    8.064942  1.924264   -0.956182    -1.220669   0.853372   -2.512958    -3.275473   0.667087   -2.219283    -2.869821   0.682933   -3.876319    -4.657703   0.504206    -1.370490     -1.826723    0.785056    -0.883366     -1.164987    0.853758
AA      59.937599         inf       NaN   0.195685    0.283158  1.035167   -0.955498    -1.204034   0.851445   -2.855830    -3.742775   0.618910   -2.629163    -3.411091   0.634198   -1.681124    -2.188681   0.743048    -1.614813     -2.092855    0.750838    -0.494562     -0.682470    0.919975
AAL     12.034462         inf       NaN  -1.415314   -1.933228  0.756436  

In [11]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1537 entries, A to ZWS
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sharpe 3d     1537 non-null   float64
 1   Sortino 3d    1537 non-null   float64
 2   Omega 3d      406 non-null    float64
 3   Sharpe 5d     1537 non-null   float64
 4   Sortino 5d    1537 non-null   float64
 5   Omega 5d      1471 non-null   float64
 6   Sharpe 10d    1537 non-null   float64
 7   Sortino 10d   1537 non-null   float64
 8   Omega 10d     1537 non-null   float64
 9   Sharpe 15d    1537 non-null   float64
 10  Sortino 15d   1537 non-null   float64
 11  Omega 15d     1537 non-null   float64
 12  Sharpe 30d    1537 non-null   float64
 13  Sortino 30d   1537 non-null   float64
 14  Omega 30d     1537 non-null   float64
 15  Sharpe 60d    1537 non-null   float64
 16  Sortino 60d   1537 non-null   float64
 17  Omega 60d     1537 non-null   float64
 18  Sharpe 120d   1537 non-null   floa

In [12]:
import numpy as np
import pandas as pd

def find_nan_inf(df):
    """
    Finds locations of NaN, Inf, and -Inf values in a DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame to check
        
    Returns:
        dict: Dictionary with column names as keys and dicts of 
              {type: indices} as values
    """
    nan_inf_locations = {}
    for col in df.columns:
        locations = {
            'NaN': df[col].index[df[col].isnull()].tolist(),
            'Inf': df[col].index[df[col] == np.inf].tolist(),
            '-Inf': df[col].index[df[col] == -np.inf].tolist()
        }
        if any(locations.values()):
            nan_inf_locations[col] = locations
    return nan_inf_locations

def print_nan_inf_locations(locations, message):
    """Prints NaN/Inf locations with a descriptive message."""
    print(message)
    if locations:
        for col, locs in locations.items():
            print(f"Column: {col}")
            for typ, indices in locs.items():
                if indices:
                    print(f"  {typ}: {indices}")
    else:
        print("No NaN or Inf values found.")

def get_column_replacement_values(df):
    """
    Calculate replacement values for each column (max for Inf/NaN, min for -Inf).
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        tuple: (max_values, min_values) dictionaries with column replacement values
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    max_values = {}
    min_values = {}
    
    for col in numeric_cols:
        # Get finite values only
        finite_vals = df[col][np.isfinite(df[col])]
        if len(finite_vals) > 0:
            max_values[col] = finite_vals.max()
            min_values[col] = finite_vals.min()
        else:
            # If no finite values, use default
            max_values[col] = 1e5
            min_values[col] = -1e5
    
    return max_values, min_values

# Check for NaN or Inf values
has_nan_inf = combined_df.isnull().any().any() or not np.isfinite(combined_df).all().all()
print(f'combined_df has_nan_inf: {has_nan_inf}')

if has_nan_inf:
    # Calculate replacement values for each column
    max_values, min_values = get_column_replacement_values(combined_df)
    print("Replacing NaN/Inf values with column-specific max/min values")
    
    nan_inf_locations_before = find_nan_inf(combined_df)
    print_nan_inf_locations(nan_inf_locations_before, "NaN/Inf Locations BEFORE Replacement:")
    
    # Create cleaned DataFrame
    df_to_save = combined_df.copy()
    
    # Replace values column by column
    for col in df_to_save.columns:
        if col in max_values:  # Only numeric columns will be in max_values
            # Replace NaN and +Inf with max value
            # NaN is from Omega ratio where daily loss series are all zero
            # +Inf is from Sharpe ratio where daily negative excess return series are all zero
            df_to_save[col] = df_to_save[col].fillna(max_values[col])
            df_to_save[col] = df_to_save[col].replace([np.inf], max_values[col])
            # Replace -Inf with min value
            # -Inf is from Sharpe ratio where daily excess return series are the same, therefore standard deviation is zero
            df_to_save[col] = df_to_save[col].replace([-np.inf], min_values[col])
else:
    print("No NaN or Inf values found.")
    df_to_save = combined_df

# Verification
print(f'df_to_save has_nan_inf: {df_to_save.isnull().any().any() or not np.isfinite(df_to_save).all().all()}')
print_nan_inf_locations(find_nan_inf(df_to_save), "NaN/Inf Locations AFTER Replacement:")

# Output cleaned DataFrame
print(df_to_save)

combined_df has_nan_inf: True
Replacing NaN/Inf values with column-specific max/min values
NaN/Inf Locations BEFORE Replacement:
Column: Sharpe 3d
  -Inf: ['BILS']
Column: Sortino 3d
  Inf: ['A', 'AA', 'AAL', 'AAON', 'AAPL', 'ABBV', 'ABEV', 'ABNB', 'ACIW', 'ACM', 'ACN', 'ACWI', 'ACWV', 'ACWX', 'ADBE', 'ADI', 'ADP', 'ADSK', 'ADT', 'AEG', 'AER', 'AES', 'AFL', 'AFRM', 'AGCO', 'AGG', 'AGNC', 'AIRR', 'AIT', 'AIZ', 'AJG', 'AKAM', 'ALAB', 'ALB', 'ALC', 'ALGN', 'ALK', 'ALL', 'ALLY', 'ALNY', 'ALSN', 'ALV', 'AMAT', 'AMD', 'AME', 'AMLP', 'AMP', 'AMX', 'AMZN', 'ANET', 'ANSS', 'AON', 'APG', 'APH', 'APO', 'APP', 'APPF', 'APTV', 'AR', 'ARCC', 'ARE', 'ARES', 'ARKB', 'ARKK', 'ARM', 'ARMK', 'ARW', 'AS', 'ASML', 'ASTS', 'ASX', 'ATI', 'ATR', 'AUR', 'AVEM', 'AVGO', 'AVLV', 'AVTR', 'AVUS', 'AVUV', 'AWI', 'AXON', 'AXP', 'AXTA', 'AYI', 'AZEK', 'AZN', 'BA', 'BABA', 'BAC', 'BAH', 'BALL', 'BAM', 'BAP', 'BAX', 'BBAX', 'BBCA', 'BBD', 'BBEU', 'BBIN', 'BBIO', 'BBJP', 'BBUS', 'BBVA', 'BBY', 'BCH', 'BCS', 'BDX', 'BECN

In [13]:
ticker_w_inf = nan_inf_locations_before['Sortino 3d']['Inf'][0]
ticker_w_inf 

'A'

#### Use Cell below to check the inf, -inf and NaN replacement

In [14]:
print(f"combined_df.loc['ticker_w_inf']")
display(combined_df.loc[ticker_w_inf])
print(f"\ndf_to_save.loc['ticker_w_inf']")
display(df_to_save.loc[ticker_w_inf])

combined_df.loc['ticker_w_inf']


Sharpe 3d       126.085003
Sortino 3d             inf
Omega 3d               NaN
Sharpe 5d         4.034383
Sortino 5d        8.064942
Omega 5d          1.924264
Sharpe 10d       -0.956182
Sortino 10d      -1.220669
Omega 10d         0.853372
Sharpe 15d       -2.512958
Sortino 15d      -3.275473
Omega 15d         0.667087
Sharpe 30d       -2.219283
Sortino 30d      -2.869821
Omega 30d         0.682933
Sharpe 60d       -3.876319
Sortino 60d      -4.657703
Omega 60d         0.504206
Sharpe 120d      -1.370490
Sortino 120d     -1.826723
Omega 120d        0.785056
Sharpe 250d      -0.883366
Sortino 250d     -1.164987
Omega 250d        0.853758
Name: A, dtype: float64


df_to_save.loc['ticker_w_inf']


Sharpe 3d        126.085003
Sortino 3d      8757.817872
Omega 3d         781.208427
Sharpe 5d          4.034383
Sortino 5d         8.064942
Omega 5d           1.924264
Sharpe 10d        -0.956182
Sortino 10d       -1.220669
Omega 10d          0.853372
Sharpe 15d        -2.512958
Sortino 15d       -3.275473
Omega 15d          0.667087
Sharpe 30d        -2.219283
Sortino 30d       -2.869821
Omega 30d          0.682933
Sharpe 60d        -3.876319
Sortino 60d       -4.657703
Omega 60d          0.504206
Sharpe 120d       -1.370490
Sortino 120d      -1.826723
Omega 120d         0.785056
Sharpe 250d       -0.883366
Sortino 250d      -1.164987
Omega 250d         0.853758
Name: A, dtype: float64

In [15]:
# Save the DataFrame to parquet file (always do this)
df_to_save.to_parquet(dest_path, engine='pyarrow', compression='zstd')
print(f"DataFrame successfully saved to {dest_path}")



DataFrame successfully saved to ..\data\2025-04-23_df_perf_ratios.parquet


In [16]:
import pandas as pd
import numpy as np
from tabulate import tabulate

_symbols = ['USFR','AAPL', 'MSFT', 'GOOG', 'NVDA', 'IBIT', 'GLD', 'TBIL']
# Create and display the formatted table
selected_df = df_to_save.loc[_symbols] #Use df_to_save here
print(tabulate(selected_df, headers='keys', tablefmt='grid', floatfmt='.4f'))

+------+-------------+--------------+------------+-------------+--------------+------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
|      |   Sharpe 3d |   Sortino 3d |   Omega 3d |   Sharpe 5d |   Sortino 5d |   Omega 5d |   Sharpe 10d |   Sortino 10d |   Omega 10d |   Sharpe 15d |   Sortino 15d |   Omega 15d |   Sharpe 30d |   Sortino 30d |   Omega 30d |   Sharpe 60d |   Sortino 60d |   Omega 60d |   Sharpe 120d |   Sortino 120d |   Omega 120d |   Sharpe 250d |   Sortino 250d |   Omega 250d |
+======+=============+==============+============+=============+==============+============+==============+===============+=============+==============+===============+=============+==============+===============+=============+==============+===============+====